<a href="https://colab.research.google.com/github/TheoLpr/NLI_study/blob/main/Natural_language_inference_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Library setup

In [ ]:
!pip3 install -q --upgrade transformers
!pip3 install -q --upgrade datasets
!pip3 install -q --upgrade evaluate

In [ ]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader, Subset

import numpy as np
from tqdm import tqdm

Use `AutoModelFor...` as this handles potential differences in model architectures in the background. Helpful so you can run many different models using practically the same code.
Similar story for `AutoTokenizer`.

In [ ]:

from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
pretrained_name_or_path = "bert-base-cased"
num_classes = 3  # {"entailment", "neutral", "contradiction"}
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
MAX_LENGTH = 128
NUM_EPOCHS = 1
LEARNING_RATE = 2e-6
BATCH_SIZE = 4
VALIDATE_EVERY_N_EXAMPLES = 10_000
# strings that should not be used in model, but might be useful for connecting predictions with examples
RAW_DATA_KEYS = {"premise", "hypothesis"}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_name_or_path)
model = AutoModelForSequenceClassification.from_pretrained(pretrained_name_or_path,
                                                           num_labels=num_classes).to(DEVICE)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)

In [ ]:
def tokenize_function(data_to_tokenize):
    return tokenizer(list(zip(data_to_tokenize["premise"],
                              data_to_tokenize["hypothesis"])),
                     max_length=MAX_LENGTH,
                     padding="max_length",
                     truncation=True,
                     return_tensors="pt")

##DATA LOADING


In [ ]:
import datasets
data = datasets.load_dataset("snli")
data = data.rename_column("label", "labels")

In [ ]:
# Drop examples with invalid labels
train_data = data["train"].filter(lambda _ex: _ex["labels"] != -1).select(range(50000))
val_data = data["validation"].filter(lambda _ex: _ex["labels"] != -1)
test_data = data["test"].filter(lambda _ex: _ex["labels"] != -1)

train_data = train_data.map(tokenize_function, batched=True, batch_size=10000, load_from_cache_file=False).with_format("torch")
val_data = val_data.map(tokenize_function, batched=True, batch_size=10000, load_from_cache_file=False).with_format("torch")
test_data = test_data.map(tokenize_function, batched=True, batch_size=10000, load_from_cache_file=False).with_format("torch")

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/9842 [00:00<?, ? examples/s]

Map:   0%|          | 0/9824 [00:00<?, ? examples/s]

In [ ]:
val_data

Dataset({
    features: ['premise', 'hypothesis', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 9842
})

In [ ]:
NUM_SUBSETS = (len(train_data) + VALIDATE_EVERY_N_EXAMPLES - 1) // VALIDATE_EVERY_N_EXAMPLES

##Training part

In [ ]:
for idx_epoch in range(NUM_EPOCHS):
    rand_indices = torch.randperm(len(train_data))
    train_loss, num_batches = 0.0, 0

    for idx_subset in range(NUM_SUBSETS):
        s_sub, e_sub = idx_subset * VALIDATE_EVERY_N_EXAMPLES, (idx_subset + 1) * VALIDATE_EVERY_N_EXAMPLES

        # Training loop
        model.train()

        # DataLoader handles things such as batching data,
        # Subset creates a smaller data subset
        for curr_batch in tqdm(DataLoader(Subset(train_data, rand_indices[s_sub: e_sub].tolist()),
                                          batch_size=BATCH_SIZE)):

            batch_on_device = {_k: curr_batch[_k].to(DEVICE) for _k in curr_batch
                            if _k not in RAW_DATA_KEYS}

            res = model(**batch_on_device)

            loss = res.loss
            train_loss += float(loss)
            num_batches += 1

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        print(f"Training loss: {train_loss / max(1, num_batches):.4f}")

        # Validation loop
        val_preds = []

        # .eval turns off dropout layers in model
        model.eval()

        # torch.inference_mode() makes sure gradients are not being computed
        # (they are only required during training and require a lot of memory)
        with torch.inference_mode():
            for curr_batch in tqdm(DataLoader(val_data, batch_size=BATCH_SIZE)):
                batch_on_device = {_k: curr_batch[_k].to(DEVICE) for _k in curr_batch
                                   if _k not in RAW_DATA_KEYS}
                res = model(**batch_on_device)

                # logits = class scores; normalize them to get probabilities
                probas = torch.softmax(res.logits, dim=-1)
                preds = torch.argmax(probas, dim=-1).cpu()
                val_preds.append(preds)

        val_preds = torch.cat(val_preds)
        # TODO: compute whatever metric you wish
        accuracy = torch.sum(val_preds == val_data["labels"]) / len(val_data["labels"])
        print(f"Validation accuracy: {accuracy:.4f}")


100%|██████████| 2500/2500 [04:52<00:00,  8.55it/s]


Training loss: 0.8911


100%|██████████| 2461/2461 [01:14<00:00, 33.03it/s]


Validation accuracy: 0.7527


100%|██████████| 2500/2500 [04:48<00:00,  8.68it/s]


...

Validation accuracy: 0.8079


100%|██████████| 2500/2500 [04:48<00:00,  8.68it/s]


Training loss: 0.6607


100%|██████████| 2461/2461 [01:14<00:00, 33.06it/s]


Validation accuracy: 0.8150


In [ ]:
val_preds[:10]


Use the HuggingFace ecosystem's built-in saving function to save your model: `<model>.save_pretrained(<path>)`. This path can be used in `.from_pretrained(<path>)` to reload the model later.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
torch.save(model, "/content/gdrive/MyDrive/Colab Notebooks/model_bert.pth")

In [ ]:
model2= torch.load("/content/gdrive/MyDrive/Colab Notebooks/model_bert.pth")

In [ ]:
train_data

##Prediction of reverse data

In [ ]:
train_data2_inv=train_data.rename_column("hypothesis","temp").rename_column("premise","hypothesis").rename_column("temp","premise").remove_columns(["labels","input_ids","attention_mask"]).map(tokenize_function, batched=True, batch_size=10000, load_from_cache_file=False).with_format("torch")
val_data2_inv=val_data.rename_column("hypothesis","temp").rename_column("premise","hypothesis").rename_column("temp","premise").remove_columns(["labels","input_ids","attention_mask"]).map(tokenize_function, batched=True, batch_size=10000, load_from_cache_file=False).with_format("torch")


Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/9842 [00:00<?, ? examples/s]

In [ ]:
train_data2_inv

Dataset({
    features: ['hypothesis', 'premise', 'token_type_ids', 'input_ids', 'attention_mask'],
    num_rows: 50000
})

In [ ]:

# Validation loop
train_preds_inv = []
val_preds_inv = []

# .eval turns off dropout layers in model
model.eval()

# torch.inference_mode() makes sure gradients are not being computed
# (they are only required during training and require a lot of memory)
with torch.inference_mode():
    for curr_batch in tqdm(DataLoader(val_data2_inv, batch_size=1)):
        batch_on_device = {_k: curr_batch[_k].to(DEVICE) for _k in curr_batch
                            if _k not in RAW_DATA_KEYS}
        res = model(**batch_on_device)

        # logits = class scores; normalize them to get probabilities
        probas = torch.softmax(res.logits, dim=-1)
        preds = torch.argmax(probas, dim=-1).cpu()
        val_preds_inv.append(preds)

with torch.inference_mode():
    for curr_batch in tqdm(DataLoader(train_data2_inv, batch_size=1)):
        batch_on_device = {_k: curr_batch[_k].to(DEVICE) for _k in curr_batch
                            if _k not in RAW_DATA_KEYS}
        res = model(**batch_on_device)

        # logits = class scores; normalize them to get probabilities
        probas = torch.softmax(res.logits, dim=-1)
        preds = torch.argmax(probas, dim=-1).cpu()
        train_preds_inv.append(preds)

100%|██████████| 50000/50000 [09:44<00:00, 85.59it/s]


In [ ]:
val_preds_inv[:10]

[tensor([0]),
 tensor([1]),
 tensor([2]),
 tensor([1]),
 tensor([2]),
 tensor([2]),
 tensor([2]),
 tensor([1]),
 tensor([1]),
 tensor([1])]

In [ ]:
from sklearn import metrics

In [ ]:
actual=list(map(int,val_data["labels"]))

predicted=list(map(int,val_preds))

confusion_matrix = metrics.confusion_matrix(actual, predicted)

#virer les tirets -> crééer une 2ème conf matrix
confusion_matrix2=np.zeros((4,3))
i=0
for ligne in confusion_matrix:
  confusion_matrix2[i]=[ligne[1],ligne[2],ligne[3]]
  i+=1

IndexError: ignored

In [ ]:
print(confusion_matrix2[1:4])


##Collect predictions as a list (easier to process)

In [ ]:
train_preds_inv2=[]
for i in range(len(train_preds_inv)):
  train_preds_inv2.append(train_preds_inv[i].tolist()[0])


val_preds_inv2=[]
for i in range(len(val_preds_inv)):
  val_preds_inv2.append(val_preds_inv[i].tolist()[0])

In [ ]:
[[x,train_preds_inv2.count(x)] for x in set(train_preds_inv2)]

[[0, 5212], [1, 20318], [2, 24470]]

In [ ]:
train_data2_inv=train_data2_inv.add_column("label",train_preds_inv2)
val_data2_inv=val_data2_inv.add_column("label",val_preds_inv2)


In [ ]:
train_data2_inv.save_to_disk("/content/gdrive/MyDrive/Colab Notebooks/train_data_inv_bert")
val_data2_inv.save_to_disk("/content/gdrive/MyDrive/Colab Notebooks/val_data_inv_bert")

Saving the dataset (0/1 shards):   0%|          | 0/50000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/9842 [00:00<?, ? examples/s]